obtain module predictions CSV file& calculate MAE (preds,gts)

In [1]:
import torch
import sys
sys.path.append("/DRIVEcodeV1/DCG_Core")
sys.path.append("your/dataset/path")
import pytorch_lightning as pl
from model import *
from module import  *
# from module_copy1 import *
# from model_copy1 import *
# from model_copy2 import *
# from main_copy1 import save_preds
from tqdm import tqdm
import os


/hpc2hdd/home/tianlangxue/anaconda3/envs/OPT/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


-----------using concept datapath at:--- /hpc2hdd/home/tianlangxue/XAI4AD/concept_gridlock/scenarios/scenarios_small_100.txt
torch.Size([100, 77])


In [2]:
import pandas as pd
def calculate_mae(actual, predicted):
    return ((actual - predicted).abs()).mean()

# 读取 CSV 文件
def read_csv_file(file_path):
    return pd.read_csv(file_path)

# 写入文件名和 MAE 到文本文件
def write_to_txt_file(txt_file_path, file_name, mae_value):
    try:
        with open(txt_file_path, 'a') as file:
            file.write(f"{file_name},{mae_value}\n")
    except Exception as e:
        print(f"Failed to write to {txt_file_path}: {e}")
        
def calculate_save_MAE(csvpathlist,textpath,task):
    
    thisbranch_MAE=[]
    txt_file_path = textpath  
    
    for csv_file_path in csvpathlist: 

        df = read_csv_file(csv_file_path)
        
        if df.shape[1] < 2:
            raise ValueError("CSV must contains 2 columns。")

        actual_values = df.iloc[:, 1]
        predicted_values = df.iloc[:, 0]

        mae = calculate_mae(actual_values, predicted_values)
        
        if task == "multitask":
            if "angle" in csv_file_path:
                txt_file_path = f"/your/analysis/record/angle_{task}_{img_noise}.txt"
            elif "dist" in csv_file_path:
                txt_file_path = f"/your/analysis/record/distance_{task}_{img_noise}.txt"
        else: txt_file_path = f"/your/analysis/record/{task}_{img_noise}.txt"
     
        file_name = csv_file_path.split('/')[-1]
     
        # print(f"MAE for {file_name}: {mae}")
   
        write_to_txt_file(txt_file_path, file_name, mae)
        thisbranch_MAE.append(mae)#a list containing 1 or 2 values(single/multi-task)
    return thisbranch_MAE

In [3]:
#checkpoints path& task setup
img_noise="GaussNoiseV3"
task = "multitask"#angle/distance/multitask ,align with version path
checkpoint ="/hpc2hdd/home/tianlangxue/XAI4AD/comma2k19data/ckpts_final/ckpts_final_comma_multitask_none_True_1/lightning_logs/version_19_testval_noreverse/checkpoints/epoch=143-step=8640.ckpt"
# ckpts_path =version_path+"/checkpoints"
# ckpt_path_list = get_ckpt_files(ckpts_path)

In [4]:
# teachmodel=TVTN(multitask=task, backbone="none", concept_features=True, device = f"cuda:0", train_concepts=False)
model = VTN(multitask=task, backbone= "none", concept_features=True, device = f"cuda:0", train_concepts=False)

using concept features


In [5]:
def save_preds(logits, target, save_name, p):
    b, s = target.shape
    df = pd.DataFrame()
    df['logits'] = logits.squeeze().reshape(b*s).tolist()
    df['target'] = target.squeeze().reshape(b*s).tolist()
    directory = os.path.dirname(f'{p}/{save_name}.csv')
    os.makedirs(directory, exist_ok=True)
    df.to_csv(f'{p}/{save_name}.csv', mode='a', index=False, header=False)

In [6]:
MAE_list=[]#save num of epochs mae results
# os.makedirs(text_save_rootpath, exist_ok=True)

testmodule = LaneModule.load_from_checkpoint(
checkpoint,
model=model,  # 传递模型参数
# teachmodel =teachmodel,
multitask=task,
dataset = "comma",
dataset_path="/hpc2hdd/home/tianlangxue/XAI4AD/comma2k19data",
bs=1,  # 传递批处理大小
ground_truth="desired",
# intervention=args.intervention_prediction,
img_noise=img_noise,
# dataset_path=args.dataset_path,
# dataset_fraction=args.dataset_fraction
)
# 定义Trainer用于预测
trainer = pl.Trainer(
    gpus=1,
    accelerator='gpu',
    devices="cuda:0",
    # logger=logger,
    # default_root_dir=ckpt_pth,  # 这个参数可以用来指定默认的输出目录
    # resume_from_checkpoint=checkpoint_path,  # 如果需要从特定的检查点恢复
    # 注：resume_from_checkpoint在预测时不常用，因为我们通常直接load_model来预测
)

preds = trainer.predict(testmodule)
#save logits into csv file


/hpc2hdd/home/tianlangxue/anaconda3/envs/OPT/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
/hpc2hdd/home/tianlangxue/anaconda3/envs/OPT/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:498: UserWarning: The flag `devices=cuda:0` will be ignored, instead the device specific number 1 will be used
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.o

using img_noise
current dataset: /hpc2hdd/home/tianlangxue/XAI4AD/comma2k19data/comma_test_w_desired_filtered_GaussNoiseV3.h5py
data_path----- /hpc2hdd/home/tianlangxue/XAI4AD/comma2k19data/comma_test_w_desired_filtered_GaussNoiseV3.h5py -----------------
Predicting DataLoader 0: 100%|██████████| 27/27 [00:10<00:00,  2.53it/s]


In [7]:
prediction_save_path ="/hpc2hdd/home/tianlangxue/XAI4AD/concept_gridlock/New/DCGPreds"
text_save_rootpath="/hpc2hdd/home/tianlangxue/XAI4AD/concept_gridlock/New/DCGMAE"
csvp=set()
print(len(preds))
assert len(preds) >= 3, "pred must have at least three elements"
for pred in preds:
    if task != "multitask":
        predictions, preds_1, preds_2 = pred[0], pred[1], pred[2] 
        save_preds(predictions, preds_1, f"single_{task}_DRIVE_100scenarios_{img_noise}", prediction_save_path)#DRIVE=concept
        csvp.add(prediction_save_path+f"/single_{task}_DRIVE_100scenarios_{img_noise}.csv")
    else:
        preds, angle, dist = pred[0], pred[1], pred[2]
        preds_angle, preds_dist = preds[0], preds[1]
        # print(preds_angle.shape,preds_dist.shape)
        # print(angle.shape,dist.shape)
        save_preds(preds_angle, angle, f"angle_multi_{task}_DRIVE_100scenarios_{img_noise}", prediction_save_path)
        save_preds(preds_dist, dist, f"dist_multi_{task}_DRIVE_100scenarios_{img_noise}", prediction_save_path)
        csvp.add(prediction_save_path+f"/angle_multi_{task}_DRIVE_100scenarios_{img_noise}.csv")
        csvp.add((prediction_save_path+f"/dist_multi_{task}_DRIVE_100scenarios_{img_noise}.csv"))
csvp =list(csvp)
print(csvp)
MAE_list.append(calculate_save_MAE(csvp,text_save_rootpath,task))

27
['/hpc2hdd/home/tianlangxue/XAI4AD/concept_gridlock/New/DCGPreds/single_distance_DRIVE_100scenarios_GaussNoiseV3.csv']
